<a href="https://colab.research.google.com/github/neuroidss/EEG-GAN-audio-video/blob/main/sg2_tf1_to_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x
#%tensorflow_version 2.x
import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__) )

TensorFlow 1.x selected.
Tensorflow version: 1.15.2


In [ ]:
%cd /content
!git clone https://github.com/NVlabs/stylegan2.git
%cd /content/stylegan2
#!git clone https://github.com/moono/stylegan2-tf-2.x.git
#%cd /content/stylegan2-tf-2.x

/content
Cloning into 'stylegan2'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 131 (delta 0), reused 1 (delta 0), pack-reused 127
Receiving objects: 100% (131/131), 593.76 KiB | 1.97 MiB/s, done.
Resolving deltas: 100% (58/58), done.
/content/stylegan2


In [ ]:
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

files_path = [['1YckI8gwqPbZBI8X4eaQAJCgWx-CqCTdi', '/content/model/sg2_anime_network-snapshot-018528.pkl']]

network_pkl=files_path[0][1]

for i in range(len(files_path)):
  gdd.download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])


if True:
  import tensorflow as tf
  import pretrained_networks
  import pprint

  #!wget --directory-prefix=./official_pretrained https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl
  #network_pkl = './official_pretrained/stylegan2-ffhq-config-f.pkl'
  print('Loading networks from "%s"...' % network_pkl)
  _G, _D, Gs = pretrained_networks.load_networks(network_pkl)
  print()

  # Print network details.
  print('===_G===')
  _G.print_layers()
  print('===Gs===')
  Gs.print_layers()

  t_var = tf.trainable_variables()
  pprint.pprint(t_var)

  saver = tf.train.Saver()

  save_path = saver.save(tf.get_default_session(), "/content/official-pretrained/model.ckpt")

In [ ]:
from google.colab import files
files.download('/content/stylegan2-tf-2.x/official-pretrained/model.ckpt.data-00000-of-00001')
files.download('/content/stylegan2-tf-2.x/official-pretrained/model.ckpt.index')
files.download('/content/stylegan2-tf-2.x/official-pretrained/model.ckpt.meta')
files.download('/content/stylegan2-tf-2.x/official-pretrained/checkpoint')
    

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp -r -v "/content/stylegan2-tf-2.x/official-pretrained" "/content/gdrive/MyDrive/EEG-GAN-audio-video/stylegan2-tf-2.x/official-pretrained"

restart 

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print('Tensorflow version: {}'.format(tf.__version__) )

Tensorflow version: 2.6.0


In [ ]:
%cd /content
!git clone https://github.com/moono/stylegan2-tf-2.x.git
%cd /content/stylegan2-tf-2.x

/content
fatal: destination path 'stylegan2-tf-2.x' already exists and is not an empty directory.
/content/stylegan2-tf-2.x


In [ ]:
%cp /content/official-pretrained/* /content/stylegan2-tf-2.x/official-pretrained/

In [ ]:
import os
import numpy as np
import tensorflow as tf

from PIL import Image
from stylegan2.utils import postprocess_images
from load_models import load_generator
from copy_official_weights import convert_official_weights_together


def test_generator(ckpt_dir, use_custom_cuda, out_fn):
    g_clone = load_generator(g_params=None, is_g_clone=True, ckpt_dir=ckpt_dir, custom_cuda=use_custom_cuda)

    # test
    seed = 6600
    rnd = np.random.RandomState(seed)
    latents = rnd.randn(1, g_clone.z_dim)
    labels = rnd.randn(1, g_clone.labels_dim)
    latents = latents.astype(np.float32)
    labels = labels.astype(np.float32)
    image_out = g_clone([latents, labels], training=False, truncation_psi=0.5)
    image_out = postprocess_images(image_out)
    image_out = image_out.numpy()

    out_fn = f'seed{seed}-{out_fn}'
    Image.fromarray(image_out[0], 'RGB').save(out_fn)
    return


def main():
    from tf_utils import allow_memory_growth

    allow_memory_growth()

    # common variables
    ckpt_dir_base = './official-converted'

    use_custom_cuda=True
    # saving phase
    for use_custom_cuda in [True, False]:
        ckpt_dir = os.path.join(ckpt_dir_base, 'cuda') if use_custom_cuda else os.path.join(ckpt_dir_base, 'ref')
        convert_official_weights_together(ckpt_dir, use_custom_cuda)

    # inference phase
    ckpt_dir_cuda = os.path.join(ckpt_dir_base, 'cuda')
    ckpt_dir_ref = os.path.join(ckpt_dir_base, 'ref')

    # 1. inference cuda saved weight from cuda model
    test_generator(ckpt_dir_cuda, use_custom_cuda=True, out_fn='from-cuda-to-cuda.png')

    # 2. inference cuda saved weight from ref model
    test_generator(ckpt_dir_cuda, use_custom_cuda=False, out_fn='from-cuda-to-ref.png')

    # 3. inference ref saved weight from ref model
    test_generator(ckpt_dir_ref, use_custom_cuda=False, out_fn='from-ref-to-ref.png')

    # 4. inference ref saved weight from cuda model
    test_generator(ckpt_dir_ref, use_custom_cuda=True, out_fn='from-ref-to-cuda.png')
    return



main()

In [ ]:
from google.colab import files
files.download('/content/stylegan2-tf-2.x/official-converted/cuda/ckpt-0.data-00000-of-00001')
files.download('/content/stylegan2-tf-2.x/official-converted/cuda/ckpt-0.index')
files.download('/content/stylegan2-tf-2.x/official-converted/cuda/checkpoint')


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

!cp -r -v "/content/stylegan2-tf-2.x/official-converted/cuda" "/content/gdrive/MyDrive/EEG-GAN-audio-video/stylegan2-tf-2.x/official-converted/cuda"

In [ ]:
  files_path = [['1Ybv8_nyDxKu-bh9ukX1Ukq-oVMSLwBma', '/content/official-pretrained/model.ckpt.data-00000-of-00001'],
              ['1nPOngJpg3hsDLtCSnHrfqG97vIzlJSG8', '/content/official-pretrained/model.ckpt.index'],  
              ['1Z5-UE1MFREuXzDllcIZwf22klcCBMgp3', '/content/official-pretrained/model.ckpt.meta'],  
              ['1q0E5i5nusP2IEH0XiEPKHwDS2foIUmmT', '/content/official-pretrained/checkpoint'],  
              ['1ngHd-Jej0rxNuL6aWglvP59X4KtxHbAp', '/content/stylegan2-tf-2.x_512/load_models.py'],  
              ['1cMzUpTL2keSVlFtG-QnIwvuKS7n7kIee', '/content/stylegan2-tf-2.x_512/inference_from_official_weights.py'],  
              ['1kjBV8DAx7m4iXK4EfL1ZTTgWGpgazGQA', '/content/stylegan2-tf-2.x_512/copy_official_weights.py'],  
              ['1SMHIeV8RTOP1-UWSW3yx8VU8lWcgPWUi', '/content/stylegan2-tf-2.x_512/inference.py']]

if generate_stylegan2_tf1:
  network_pkl=files_path[0][1]

for i in range(len(files_path)):
  gdd.download_file_from_google_drive(file_id=files_path[i][0], dest_path=files_path[i][1])

if generate_stylegan2_tf2:
  %cp /content/official-pretrained/* /content/stylegan2-tf-2.x/official-pretrained/
  %cp /content/stylegan2-tf-2.x_512/* /content/stylegan2-tf-2.x/
